In [1]:
import pandas as pd 
df = pd.read_parquet('cmg-data-v2\\cmg-data-cleaned.parquet',engine='fastparquet')
test_index = list()
with open('cmg-data-v2\\split-data\\cross_project\\test_id.txt') as f:
    test_index  =[l.strip() for l in f.readlines()]
df = df[df['index'].isin(test_index)]
df.shape
import re
results = list()
for idx in test_index:
    source_seq = ''
    target_seq = ''
    for _, row in df[df['index']==idx].iterrows():
        diffs = list()
        for l in row['diff'].splitlines():
            l = re.sub('@@.+?@@', '', l)
            l = re.sub(r'\s+', ' ', l)
            if len(l) <= 0:
                continue
            words = l.split()
            diffs.append(' '.join(words))
            
        if row['old_path_file'] != None:
            old_f = row['old_path_file'].split()
            source_seq += 'mmm ' + ' '.join(old_f) + ' <s> '
        
        if row['new_path_file'] != None:
            new_f = row['new_path_file'].split()
            source_seq += 'ppp ' + ' '.join(new_f) + ' <s> '
        
        source_seq += ' \n '.join(diffs)
        label_words = row['label'].split()
        target_seq = ' '.join(label_words)
    results.append({'index':idx,'source':source_seq,'label':target_seq})

print(len(results))
import json
with open('test_cross_project_patch.jsonl','w+') as f:
    for obj in results:
        f.writelines(json.dumps(obj)+"\n")

5952


In [2]:
import json
import os
with open('test_cross_project_patch.jsonl') as f:
    source = list()
    for l in f.readlines():
        obj = json.loads(l.strip())
        source.append(obj)
output_dir = 'data/patch_openai'
queue_api = list()
PROMPT_TEMPLATE = """
I want you to act as a commit message generator.
Do not write any explanations or other words, just reply with the commit message.
Please generate the message of this changes:
%s
"""
for obj in source:
    prompt = PROMPT_TEMPLATE.strip() % obj['source']
    post_api = {"custom_id":obj['index'],
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                "model": "gpt-3.5-turbo-0125",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0,
                "max_tokens": 256}}
    queue_api.append(post_api)
    

In [4]:
with open('queue_api_patch_cross_project.jsonl','w+') as f:
    for obj in queue_api:
        f.writelines(json.dumps(obj)+"\n")

**SPLIT batch**

In [9]:
from transformers import RobertaTokenizer
import json
MAX = 170000
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
file_prefix = 'mini_batch'

import glob
for file_process in glob.glob('human_eval\\open_ai_call\\*.jsonl'):
    # file_process = 'human_eval\\open_ai_call\\java_method_name.jsonl'
    file_name = file_process.split("\\")[-1].split('.')[0]
    print(file_name)
    index = 0
    count = 0
    result = list()
    root = "human_eval\\open_ai_call\\"
    with open(file_process) as f:
        for line in f.readlines():
            obj = json.loads(line.strip())
            content = obj['body']['messages'][0]['content']
            tokens = tokenizer.tokenize(content)
            if count + len(tokens) >MAX:
                with open(f"{root}{file_prefix}_{index}_{file_name}.jsonl",'w+') as ff:
                    for tmp in result:
                        ff.writelines(json.dumps(tmp) + '\n')
                print(len(result))
                count = len(tokens)
                result = [obj]
                index += 1
            else:
                result.append(obj)
                count += len(tokens)
    with open(f"{root}{file_prefix}_{index}_{file_name}.jsonl",'w+') as ff:
        for tmp in result:
            ff.writelines(json.dumps(tmp) + '\n')
    print(len(result))

java_control_dep
158
java_data_dep
158
java_method_name
969
453
java_output
914
806
740
743
691
876
652
537
778
764
804
801
782
30
java_summarize
983
439
python_control_dep
164
python_data_dep
164
python_method_name
1476
python_output
1433
1459
1112
1208
1230
870
938
1149
447
python_summarize
1476
